In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
# !wget https://huggingface.co/datasets/mesolitica/malaysian-news/resolve/main/news-2022-11-18.json

In [3]:
import json
import malaya
from tqdm import tqdm
from unidecode import unidecode

In [4]:
import re

def cleaning(string):
    splitted = malaya.text.function.split_into_sentences(string)
    if not len(splitted):
        splitted = '. '.join([k.strip() for k in string.split('.') if len(k.strip())])
    if splitted[0][0] == '-':
        splitted[0] = splitted[0].replace('- ','')
    points = [f'{no + 1}. {s}' for no, s in enumerate(splitted)]
    points = ' '.join(points)
    return points

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

In [5]:
!tail -n 2 populate-news.json.semisupervised

{"title": "Pelancong ke M'sia nak tengok Mat, bukan Doraemon", "url": "https://www.malaysiakini.com/letters/473507", "authors": [], "top-image": "https://i.malaysiakini.com/777/13e94b92eae8996a850667bb8c2c06a2.jpg", "text": "Saya pertama kali menonton watak Doraemon di televisyen sekitar tahun 1993, ketika sedang dalam latihan amali di sebuah kilang di Sungai Way.\n\nSaya dalam satu rombongan ke Cameron Highlands bersama-sama pekerja kilang dan kami menginap di sebuah banglo di sana. Sebaik sampai, kami berehat dan menonton televisyen beramai-ramai di ruang tamu dan itulah pertama kali menyaksikan watak kartun itu. Sebelumnya, saya hanya mendengar dari cakap-cakap orang sahaja.\n\nMaknanya, Doraemon ini sudah masuk ke negara kita begitu lama. Sebelum tahun 1993, ianya telah bertapak sebagai satu watak kegemaran kanak-kanak (dan sebilangan orang dewasa juga).\n\nIa seperti juga watak Ultraman, telah menjadi topik perbualan sejak dari waktu itu lagi. Menariknya, Ultraman telah bertukar g

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from malaya.text.vectorizer import SkipGramCountVectorizer

stopwords = malaya.text.function.get_stopwords()
bow = CountVectorizer(
    ngram_range = (1, 4),
    stop_words = stopwords,
    lowercase = False,
)

stopwords = malaya.text.function.get_stopwords()
skip_bow = SkipGramCountVectorizer(
    ngram_range = (1, 4),
    stop_words = stopwords,
    lowercase = False,
    skip = 2
)

In [7]:
before, after = [], []

count = 0
rejected = []
languages = []
accepted = []
para = []
malaysian_news = {
    'kosmo',
    'hmetro',
    'malaymail',
    'projekmm',
    'bharian',
    'utusan',
    'astroawani',
    'themalaysianinsight',
    'malaysiakini',
    'bernama'
}

def reject(data):
    if data['news'] in malaysian_news:
        return False
    if any([n in data['top-image'] for n in malaysian_news]):
        return False
    if any([n in data['url'] for n in malaysian_news]):
        return False
    if 'com.my' in data['top-image']:
        return False
    if data['language'] == 'malay':
        return False
    if 'Siaran Pers' in data['news']:
        return True
    if '.id' in data['news']:
        return True
    
    return True

with open('populate-news.json.semisupervised') as fopen:
    for l in tqdm(fopen):
        data = json.loads(l)
        text = re.sub(r'[ ]+', ' ', data['text']).strip()
        if 'kindly register' in text.lower() or 'disabled in your browser' in text.lower():
            continue
        if len(text.split()) < 30:
            rejected.append(data)
            continue
            
        if data['language'] == 'ENGLISH':
            continue
            
        accepted.append(data)
            
        
#         befores = data['semisupervised-summaries']
#         for b in befores:
#             b = cleaning(b)
#             if len(b.split()) < 3:
#                 continue
#             before.append(b)
#             after.append(text)
            
#         if count == 10:
#             break
            
        count += 1

81717it [00:05, 15792.67it/s]


In [8]:
len(accepted)

79560

In [9]:
accepted[0]

{'title': 'Ibu saudara Haziq terharu sokongan rakyat Malaysia',
 'url': 'https://www.themalaysianinsight.com/bahasa/s/142491',
 'news': 'themalaysianinsight',
 'language': 'malay',
 'top-image': 'https://www.themalaysianinsight.com/resources/stories_images/142491/perhimpunanan_solidarity_kedamaian_03__full.jpg',
 'text': 'WARIS keluarga Allahyarham Muhammad Haziq Mohd Tarmizi, 17, yang terkorban dalam tragedi tembakan di Christchurch, New Zealand, pada 15 Mac lepas, melahirkan rasa terharu akan sokongan diberi rakyat Malaysia semasa perhimpunan Solidariti Kedamaian.\n\nZarina Shuib , ibu saudara Muhammad Haziq, memanjatkan kesyukuran kepada Allah SWT kerana berkesempatan menyertai rakyat Malaysia dalam perhimpunan itu di Kuala Lumpur hari ini.',
 'date': '2019-03-23T03:52:02',
 'date_utc': '2019-03-22T19:52:02',
 'semisupervised-summaries': ['Perhimpunan Solidariti Kedamaian () adalah perhimpunan solidariti yang diadakan di Kuala Lumpur, Malaysia, pada 15 Mac 2013.',
  'WARIS keluarga 

In [60]:
rejected = {'saya', 'awak', 'kami', 'dia', 'mereka', 'anda', 'kita'}

tokenize = malaya.tokenizer.Tokenizer().tokenize

def headline(string, length = 300):
    splitted = malaya.text.function.split_into_sentences(string)
    splitted = [s for s in splitted if all([r not in s for r in rejected])]
    selected, index, results = '', 0, []
    while len(selected) <= length and index < len(splitted):
        selected += splitted[index]
        results.append(splitted[index])
        index += 1
    return ' '.join(results)

def reject(string):
    splitted = malaya.text.function.split_into_sentences(string)
    splitted = [s for s in splitted if not len(set(tokenize(s.lower())) & rejected)]
    return ' '.join(splitted)

In [57]:
s = 'IBU negara Korea Selatan, Seoul menduduki tempat ketujuh dalam senarai bandar raya paling mahal untuk didiami pada tahun ini, lapor agensi berita Yonhap.'

In [58]:
reject(accepted[i]['text'])

['KUALA LUMPUR, 26 Mac (Bernama) -- Malaysia akan sentiasa bertegas dan vokal dalam isu yang melibatkan penindasan ke atas etnik Rohingya di Myanmar dan konsisten untuk membantu pelarian tersebut daripada terus dinafikan hak mereka, kata Menteri Luar Datuk Saifuddin Abdullah.', 'Beliau berkata tiga perkara utama yang akan diteruskan kerajaan ialah menegakkan keadilan dan membawa mereka yang terlibat dalam pelanggaran hak asasi manusa itu ke pengadilan dan diadili mengikut undang-undang antarabangsa, pemberian status kewarganegaraan Myanmar kepada etnik Rohingya dan membawa pulang mereka yang melarikan diri ke Bangladesh untuk pulang ke Myanmar".', 'Malaysia memainkan peranan yang besar khususnya di Hospital Cox\'s Bazar, Bangladesh dan kita juga membantu mereka di dalam negara".', 'Semuanya telah kita laksanakan tetapi isu ini masih juga belum selesai.', 'Kita bimbang jika ia tidak diselesaikan, ia semakin berpanjangan dan lebih ramai (Etnik Rohingya di Myanmar) yang akan melarikan dir

'KUALA LUMPUR, 26 Mac (Bernama) -- Malaysia akan sentiasa bertegas dan vokal dalam isu yang melibatkan penindasan ke atas etnik Rohingya di Myanmar dan konsisten untuk membantu pelarian tersebut daripada terus dinafikan hak mereka, kata Menteri Luar Datuk Saifuddin Abdullah. Beliau berkata tiga perkara utama yang akan diteruskan kerajaan ialah menegakkan keadilan dan membawa mereka yang terlibat dalam pelanggaran hak asasi manusa itu ke pengadilan dan diadili mengikut undang-undang antarabangsa, pemberian status kewarganegaraan Myanmar kepada etnik Rohingya dan membawa pulang mereka yang melarikan diri ke Bangladesh untuk pulang ke Myanmar". Malaysia memainkan peranan yang besar khususnya di Hospital Cox\'s Bazar, Bangladesh dan kita juga membantu mereka di dalam negara". Semuanya telah kita laksanakan tetapi isu ini masih juga belum selesai. Kita bimbang jika ia tidak diselesaikan, ia semakin berpanjangan dan lebih ramai (Etnik Rohingya di Myanmar) yang akan melarikan diri dan lebih r

In [13]:
import random

In [14]:
keywords = malaya.keyword.extractive.rake(accepted[2]['text'], 
                                                      vectorizer = bow, 
                                                      top_k = random.randint(3, 10))
keywords = [k[1] for k in keywords]
keywords_rake = malaya.keyword.extractive.rake(accepted[2]['text'], atleast = 1,
                                          top_k = random.randint(3, 10))
keywords_rake = [k[1] for k in keywords_rake]
keywords, keywords_rake

(['Hospital Pulau Pinang mangsa',
  'Pulau Pinang mangsa menafikan',
  'mangsa laporan polis mangsa',
  'kelmarin Christopher individu nama'],
 ['Ketua Polis Daerah Barat Daya',
  '000 menerusi nombor akaun CIMB',
  'individu nama Lee Song',
  'mangsa menafikan mengenali individu',
  'nama Christopher Anak Anor',
  'disambungkan menerusi telefon',
  'PULAU PINANG 23 Mac'])

In [15]:
months = {
    'january',
    'jan',
    'januari',
    'february',
    'feb',
    'februari',
    'march',
    'mac',
    'april',
    'apr',
    'may',
    'mei',
    'june',
    'jun',
    'july',
    'julai',
    'august',
    'ogos',
    'aug',
    'september',
    'sep',
    'october',
    'oktober',
    'oct',
    'november',
    'nov',
    'december',
    'disember',
    'dec',
    'utusan',
    'malaysiakini',
    'astroawani',
    'bernama',
    'com',
}

In [16]:
from malaya.text.rouge import postprocess_summary, filter_rouge, _get_word_ngrams, _rouge_clean, cal_rouge
from malaya.text.function import split_into_sentences

def filtering_rouge(summary, contents, start = 0.15, increment = 0.05, break_at = 120, n = 1):
    filtered = contents[:]
    i = 0
    while len(filtered.split()) > break_at:
        filtered = filter_rouge(summary, filtered, n = n, threshold = start + increment * i)
        i += 1
    return filtered

In [17]:
i = 2
keywords = malaya.keyword.extractive.rake(accepted[i]['text'], 
                                                  vectorizer = bow, 
                                                  top_k = 50)
keywords = [simple_cleaning(k[1]) for k in keywords if len(k[1].split()) > 1 and len(k[1]) > 10 \
                and len(set(k[1].lower().replace('-', '').split()) & months) == 0]
keywords = keywords[:random.randint(3, 10)]
keywords, 

(['Hospital Pulau Pinang mangsa',
  'Pulau Pinang mangsa menafikan',
  'mangsa laporan polis mangsa',
  'kelmarin Christopher individu nama'],)

In [61]:
before, after = [], []
for i in tqdm(range(len(accepted))):
    h = reject(accepted[i]['text'])
    try:

        keywords = malaya.keyword.extractive.rake(h, vectorizer = bow, top_k = 100)
        keywords = [simple_cleaning(k[1]) for k in keywords if len(k[1].split()) > 1 and len(k[1]) > 10 \
                        and len(set(k[1].lower().replace('-', '').split()) & months) == 0]
        keywords = keywords[:random.randint(3, 10)]
        
        r = filtering_rouge('. '.join(keywords), h, n = 2, start = 0.01)
        
        if len(r.split()) > 20:
            before.append(keywords)
            after.append(r)
        
        keywords_rake = malaya.keyword.extractive.rake(h, top_k = 100)
        keywords_rake = [simple_cleaning(k[1]) for k in keywords_rake if len(k[1].split()) > 1 and len(k[1]) > 10 \
                        and len(set(k[1].lower().replace('-', '').split()) & months) == 0]
        keywords_rake = keywords_rake[:random.randint(3, 10)]
        r = filtering_rouge('. '.join(keywords_rake), h, n = 2, start = 0.01)
        
        if len(r.split()) > 20:
            before.append(keywords_rake)
            after.append(r)
        
    except Exception as e:
        # print(h, accepted[i]['text'])
        print(e)

  0%|▏                                     | 274/79560 [00:02<10:00, 132.05it/s]

empty vocabulary; perhaps the documents only contain stop words


  0%|▏                                     | 381/79560 [00:03<10:40, 123.55it/s]

empty vocabulary; perhaps the documents only contain stop words


  1%|▎                                     | 608/79560 [00:04<09:35, 137.28it/s]

...

 97%|███████████████████████████████████▋ | 76779/79560 [14:23<00:28, 96.07it/s]

empty vocabulary; perhaps the documents only contain stop words


 99%|████████████████████████████████████▋| 78945/79560 [14:48<00:06, 97.15it/s]

empty vocabulary; perhaps the documents only contain stop words


100%|█████████████████████████████████████| 79560/79560 [14:55<00:00, 88.85it/s]


In [62]:
len(before), len(after)

(155967, 155967)

In [64]:
before[100], after[100]

(['Perdana Menteri Pakistan Imran Khan menyifatkan rakan sejawatnya Pertana Menteri Malaysia',
  'krisis ekonomi melanda Asia Tenggara',
  'meningkatkan taraf hidup rakyatnya',
  'tenaga kerja warganegara Pakistan',
  'isu-isu membabitkan dunia Islam',
  'pengganas menembak mati',
  'peluang-peluang pekerjaan diwujudkan',
  'umat Islam dilayan',
  '3 bilion umat Islam',
  'menjejaskan dunia Islam'],
 ': Perdana Menteri Pakistan Imran Khan menyifatkan rakan sejawatnya Pertana Menteri Malaysia Tun Dr Mahathir Mohamad sebagai negarawan Islam yang telah merubah Malaysia menjadi sebuah model teladan untuk dunia Islam. Beliau berkata Pakistan berbangga dengan cara Malaysia membangun dan meningkatkan taraf hidup rakyatnya di bawah kepimpinan Dr Mahathir". Seperti memerangi keganasan, bagaimana ia telah menjejaskan dunia Islam. Bagaimana umat Islam dilayan disebabkan perjuangan politik yang mengelirukan dan disengajakan". Kini tenaga kerja warganegara Pakistan sudah mula pergi ke Malaysia... p

In [65]:
with open('keywords-headline.json', 'w') as fopen:
    json.dump({'before': before, 'after': after}, fopen)